# Writing files

In [1]:
import your

your.__version__

'0.6.2'

In [2]:
from your import Your, Writer
import os
import numpy as np
import tempfile
from urllib.request import urlretrieve

In [3]:
import logging

logger = logging.getLogger()
logger = logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(threadName)s - %(levelname)s -" " %(message)s",
)

We can write + convert files using the [Writer](https://thepetabyteproject.github.io/your/writer/#writer) class in [Your](https://thepetabyteproject.github.io/your/your/#your). Let's read a Filterbank file and write out a PSRFITS and a Filterbank file

In [4]:
temp_dir = tempfile.TemporaryDirectory()
download_path = str(temp_dir.name) + "/FRB180417.fil"
url = "https://zenodo.org/record/3905426/files/FRB180417.fil"
urlretrieve(
    url, download_path,
)
fil_file = download_path
your_original_file = Your(fil_file)

While writing files you can also select the required channel and spectra range. Just give the required ranges as input to the Writer class.  

In [12]:
writer_object = Writer(
    your_original_file,
    nstart=0,
    nsamp=10,
    c_min=10,
    c_max=100,
    outdir="."
    outname="filfromfil",
)

## Writing Filterbanks

To write a filterbank we use the [`to_fil`](https://thepetabyteproject.github.io/your/writer/#to_fil) method,

In [13]:
writer_object.to_fil()

Output()

We can just read this Filterbank using [Your](https://thepetabyteproject.github.io/your/your/#your) to make sure that the header parameters and the data are correctly written.

In [14]:
your_written_filterbank = Your("filfromfil.fil")
your_written_filterbank.your_header

OSError: filfromfil.fil not found

We can also compare data from both the original file and the newly written file:

In [ ]:
original_data = your_original_file.get_data(0, 10)[:, 10:100]
filterbank_data = your_written_filterbank.get_data(0, 10)

In [ ]:
np.array_equal(original_data, filterbank_data)

Looks good!

## Writing PSRFITS



All the inputs defined while declaring Writer are still gonna be used. Let's change the outname for clarity
and select some channels

In [ ]:
writer_object = Writer(
    your_original_file,
    nstart=0,
    nsamp=10,
    c_min=0,
    c_max=100,
    outdir=".",
    outname="fitsfromfil",
)

In [ ]:
writer_object.outname = "fitsfromfil"
writer_object.c_min = 0
writer_object.c_max = 100

To write a PSRFITS we use the [`to_fits`](https://thepetabyteproject.github.io/your/writer/#to_fits) method,

In [ ]:
writer_object.to_fits()

Similar to reading the PSRFITS file, we can also read a filterbank file and convert to both formats by first defining a [`writer_object`](https://thepetabyteproject.github.io/your/writer/) and using the [`to_fits`](https://thepetabyteproject.github.io/your/writer/#to_fits) and [`to_fil`](https://thepetabyteproject.github.io/your/writer/#to_fil) methods.

## Writing Filterbanks from scratch

First step in writing a filterbank is to create the header. We will use the [`make_sigproc_object`](https://thepetabyteproject.github.io/your/formats/filwriter/#make_sigproc_object) function for this.

In [ ]:
import numpy as np
from your.formats.filwriter import make_sigproc_object

In [ ]:
sigproc_object = make_sigproc_object(
    rawdatafile  = "foo.fil",
    source_name = "bar",
    nchans  = 1024, 
    foff = -0.1, #MHz
    fch1 = 2000, # MHz
    tsamp = 256e-6, # seconds
    tstart = 59246, #MJD
    src_raj=112233.44, # HHMMSS.SS
    src_dej=112233.44, # DDMMSS.SS
    machine_id=0,
    nbeams=0,
    ibeam=0,
    nbits=8,
    nifs=1,
    barycentric=0,
    pulsarcentric=0,
    telescope_id=6,
    data_type=0,
    az_start=-1,
    za_start=-1,
)

Now we have `sigproc_object` which is an instance of the [`SigprocFile`](https://thepetabyteproject.github.io/your/formats/pysigproc/#sigprocfile) class. The class contains all the required functions to write a new file.

First we use [`write_header`](https://thepetabyteproject.github.io/your/formats/pysigproc/#write_header) to write the header in `foo.fil` and then we use [`append_spectra`](https://thepetabyteproject.github.io/your/formats/pysigproc/#append_spectra) to fill data in the file.

In [ ]:
sigproc_object.write_header("foo.fil")

Now let's create some random data with 8192 time samples and 1024 frequency channels to fill in our file and then append it.

In [ ]:
data_written = np.random.randint(low=0, high=255, size=(8192,1024), dtype=np.uint8)

In [ ]:
sigproc_object.append_spectra(data_written, "foo.fil")

We can test if the data written is correct by opening it using Your.

In [ ]:
your_filterbank = Your("foo.fil")

In [ ]:
your_filterbank.your_header

In [ ]:
data_read = your_filterbank.get_data(nstart=0, nsamp=8192)

In [ ]:
np.array_equal(data_read, data_written)